# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [8]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

605

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).

> **HINT:** The OpenWeatherMap API only allows 60 calls per minute under their free plan. Try using `time.sleep(60)` after each set of 50 cities to avoid API errors due to large calls.

In [13]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"
units="imperial"
#Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

lat = []
lng = []
max_temp =[]
humidity=[]
cloudiness=[]
wind_speed=[]
country=[]
date=[]
cities1=[]

for city in cities:
    try:
        response = requests.get(query_url+city).json()
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        max_temp.append(response['main']['temp_max'])
        humidity.append(response['main']['humidity'])
        cloudiness.append(response['clouds']['all'])
        wind_speed.append(response['wind']['speed'])
        country.append(response['sys']['country'])
        date.append(response['dt'])
        cities1.append(city)
        print(f"city processed: {city}")
    except:
        print(f"-----------city NOT found: {city}")
print("----------------------------------\nData Retrieval Complete\n----------------------------------")

city processed: buin
city processed: georgetown
city processed: tampin
city processed: mataura
city processed: chuy
city processed: ushuaia
city processed: hilo
-----------city NOT found: illoqqortoormiut
city processed: butaritari
-----------city NOT found: taolanaro
city processed: payakumbuh
city processed: pevek
city processed: pasinler
city processed: surt
city processed: albany
city processed: muzhi
city processed: kismayo
city processed: torbay
city processed: bredasdorp
city processed: dikson
city processed: coquimbo
city processed: kaeo
-----------city NOT found: saleaula
city processed: nikolskoye
city processed: hami
city processed: paamiut
city processed: hermanus
city processed: the valley
city processed: carnarvon
city processed: busselton
city processed: saldanha
city processed: port hardy
city processed: punta arenas
city processed: qaanaaq
city processed: tralee
city processed: kapaa
city processed: port alfred
city processed: hobart
city processed: puerto ayora
city p

city processed: san policarpo
city processed: yabelo
city processed: jacksonville
city processed: novosergiyevka
city processed: chokurdakh
city processed: port blair
city processed: kuito
city processed: port augusta
city processed: jaisinghnagar
city processed: cidreira
city processed: kahului
city processed: shimanovsk
-----------city NOT found: grand river south east
city processed: tecoanapa
city processed: port hedland
city processed: businga
city processed: tandil
city processed: salinopolis
city processed: rawson
city processed: tazovskiy
city processed: pulsano
city processed: lakes entrance
city processed: veraval
city processed: nhulunbuy
city processed: kasangulu
city processed: viedma
city processed: praia da vitoria
city processed: strezhevoy
city processed: auki
city processed: korla
-----------city NOT found: bargal
-----------city NOT found: marcona
city processed: la ligua
city processed: san antonio
city processed: bathsheba
city processed: orange cove
city processed

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [14]:
# Create a dataframe
weather_dict = {
    "City":cities1,
    "Lat":lat,
    "Lng":lng,
    "Max Temp":max_temp,
    "Humidity":humidity,
    "Cloudiness":cloudiness,
    "Wind Speed":wind_speed,
    "Country":country,
    "Date":date
}
weather_df = pd.DataFrame(weather_dict)
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,buin,-33.7333,-70.7500,71.87,70,0,1.23,CL,1643255211
1,georgetown,5.4112,100.3354,87.73,76,20,5.01,MY,1643255163
2,tampin,2.4701,102.2302,87.33,58,91,7.29,MY,1643255212
3,mataura,-46.1927,168.8643,57.47,41,44,7.96,NZ,1643255213
4,chuy,-33.6971,-53.4616,69.87,86,92,18.19,UY,1643255213
...,...,...,...,...,...,...,...,...,...
557,caravelas,-17.7125,-39.2481,74.03,93,67,3.38,BR,1643255458
558,kosino,58.4127,51.2782,4.39,97,99,4.99,RU,1643255459
559,karatau,43.1667,70.4667,27.93,51,0,3.78,KZ,1643255459
560,buenavista,8.9769,125.4089,86.43,68,26,8.63,PH,1643255460


In [17]:
# Export the data into a .csv
weather_df.to_csv(r'C:/Users/foh01/DataClass/6.-Python-API-Challenge/WeatherPy/output_data/weather.csv', index=False)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [19]:
humidity_check = weather_df[(weather_df['Humidity']>100)]
humidity_check

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


In [20]:
#  Get the indices of cities that have humidity over 100%.
humidity_check.index

Int64Index([], dtype='int64')

In [2]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

###There are no cities that have humidity > 100%

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression